In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np 
import pandas as pd

Using TensorFlow backend.


In [0]:
tokenizer = Tokenizer()

In [0]:
data_1 = "Hollywood’s superhero extravaganza Avengers Endgame continues its dream run in India on the sixth day of its release, with the Marvel film earning an estimated Rs 25 crore on Wednesday, bringing its total box office collection to Rs 240 crore at domestic ticket windows. With Wednesday’s earnings, the film is inching closer to Rs 250-mark in India and ready to breach Vicky Kaushal’s Uri The Surgical Strike’s box office collection, which earned Rs 244 crore. Avengers: Endgame has earned $1.05 billion globally and is expected to cross the $2 billion mark this weekend. On Tuesday, the film eclipsed the lifetime earnings of Black Panther ($1.347 billion) and Avengers: Age Of Ultron ($1.405 billion) to emerge as the eighth highest grossing global movie of all time. It also went past the records of Harry Potter And The Deathly Hallows Part 2 ($961 million), The Fate Of The Furious ($1.010 billion) and Jurassic World ($1.019 billion) to emerge as the sixth highest earner globally. It is expected to outdo Titanic’s lifetime collections but the world is watching closely whether it can go beyond Avatar to become the highest earner ever. As per a Deadline report, India ranked among top five markets for the movie on Tuesday. The report also stated the Tuesday’s box office collection figures, which put China on top with $389.1 million earnings, with UK and Korea closely following with $64.7 million and $54.8 million collection. While the film earned $40.7 million in Mexico, it collected $36.3 million in India. The film has been breaking box office records ever since it released on April 26. Avengers Endgame is the highest Hollywood opener in India and also eclipsed the biggest opening weekend of Hindi films ever. Following Bahubali 2 The Conclusion’s Rs 121 crore opening, Avengers Endgame is also the biggest opener ever in India. Worldwide, the film became biggest opener ever and fastest movie ever to reach $1 billion mark. The MCU film hit the milestone in five days – it released in many countries on Wednesday – while Avengers: Infinity War took 11 days to reach $1 billion mark. It also became the highest earning film in China with $330.5 million in five days. Directed by Joe Russo and Anthony Russo, Avengers: Endgame features Robert Downey Jr, Chris Evans, Mark Ruffalo and Brie Larson. It is a direct sequel to Avengers: Infinity War and is the culmination of the previous 20-plus titles in the Marvel Cinematic Universe (MCU). Avengers Endgame’s box office collection has been an eye-opener. Earlier, Endgame’s predecessor, Avengers: Infinity War (2018) held the highest opening weekend record with Rs 94.30 crore. “It should be an eye-opener. If Endgame can do such business, why can’t our films do the same? It shows the kind of potential Indian film market has. As long as a film is entertaining and engaging, audiences will lap it up. Are we providing our audiences such big-ticket world class entertainers that can do business in a similar fashion? No, we aren’t,” says trade analyst Taran Adarsh. Filmmakers also concur. Says Sanjay Gupta: “Wherever the audiences get a quality product, they will go for it. Now, the writing is on the wall that you either perform or perish.” Filmmaker Shekhar Kapur tweeted: Bollywood needs to up its game... Now we have no choice but to compete with Hollywood. Be more internationally competitive. Songs/dances will get us just so far. Need to go beyond. "

In [0]:
data_2 = "Avengers: Endgame has become the highest grossing Hollywood film in India, overtaking its predecessor Avengers: Infinity War’s lifetime earnings in less than a week. The Marvel superhero film has made Rs 244 crore (nett) in six days, and Rs 290 crore gross. Infinity War had concluded its run with around Rs 230 crore nett. Trade analyst Taran Adarsh called the performance an ‘eye-opener’ and said that the film is competing with the biggest Hindi films of all time, and could challenge Dangal’s lifetime business as well. The Aamir Khan-starrer is the highest grossing Hindi film of all time. Endgame has already posted the biggest opening day and biggest opening weekend for a Hollywood film in India, and the number two opening of all time, behind Baahubali: The Conclusion. The film was released in 2800 screens, less than half of Baahubali 2’s 6500 screens. “#AvengersEndgame is competing with the biggest hits from *Hindi* film industry... Has stamina to cross *lifetime biz* of #Dangal,” wrote Taran on Twitter. He continued, “#AvengersEndgame is the highest grossing film of 2019 so far.” The film has overtaken the likes of Uri: The Surgical Strike, Gully Boy, Total Dhamaal and others. Taran wrote that every future Hindi release will be compared to Avengers: Endgame. Internationally, the film is on track to challenge Avatar’s all-time record, having already broken the opening weekend records in multiple territories. The film made an unprecedented $1.2 billion in its first three days, almost double that of Infinity War. The film’s current worldwide tally is $1.4 billion."

In [0]:
content = data_1+data_2

In [0]:
len(content.lower().split("."))

65

In [0]:
corpus =content.lower().split(".")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
total_words

365

In [0]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
len(input_sequences)

793

In [0]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)
(input_sequences[:,-1]).shape

(793,)

In [0]:
max_sequence_len

44

In [0]:
input_sequences
label.shape

(793, 365)

In [0]:
predictors.shape

(793, 43)

In [0]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences = True))
# model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences= True))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
model.fit(predictors, label, epochs=100, verbose=1, callbacks=[earlystop])
print (model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
793/793 [==============================] - 8s 10ms/step - loss: 5.7713 - acc: 0.0530
Epoch 2/100
 32/793 [>.............................] - ETA: 4s - loss: 5.2485 - acc: 0.1250

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


793/793 [==============================] - 4s 5ms/step - loss: 5.4580 - acc: 0.0618
Epoch 3/100
793/793 [==============================] - 4s 6ms/step - loss: 5.3710 - acc: 0.0618
Epoch 4/100
793/793 [==============================] - 4s 6ms/step - loss: 5.3397 - acc: 0.0618
Epoch 5/100
793/793 [==============================] - 4s 6ms/step - loss: 5.3307 - acc: 0.0618
Epoch 6/100
793/793 [==============================] - 4s 6ms/step - loss: 5.3258 - acc: 0.0618
Epoch 7/100
793/793 [==============================] - 4s 6ms/step - loss: 5.3216 - acc: 0.0618
Epoch 8/100
793/793 [==============================] - 4s 6ms/step - loss: 5.3203 - acc: 0.0618
Epoch 9/100
793/793 [==============================] - 4s 5ms/step - loss: 5.3199 - acc: 0.0618
Epoch 10/100
793/793 [==============================] - 4s 5ms/step - loss: 5.3173 - acc: 0.0618
Epoch 11/100
793/793 [==============================] - 5s 6ms/step - loss: 5.3163 - acc: 0.0618
Epoch 12/100
793/793 [============================

In [0]:
def generate_text(seed_text, next_words, max_sequence_len):
	for _ in range(next_words):
		token_list = tokenizer.texts_to_sequences([seed_text])[0]
		token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
		predicted = model.predict_classes(token_list, verbose=0)
		
		output_word = ""
		for word, index in tokenizer.word_index.items():
			if index == predicted:
				output_word = word
				break
		seed_text += " " + output_word
	return seed_text

In [0]:
print (generate_text("Avengers", 14, max_sequence_len))

Avengers endgame has earned 1 the milestone 1 the war’s lifetime earnings in less than
